In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import average_precision_score
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
from sklearn import tree
from sklearn.model_selection import cross_val_score

import lime
import lime.lime_tabular

import holoviews as hv

In [2]:
def read_csv(filepath):
    data = pd.read_csv(filepath)
    data.head()
    return data

In [3]:
def split_to_2_predictors(dataset1, dataset2, features):
    predictor1 = dataset1[features]
    predictor2 = dataset2[features]
    return predictor1, predictor2

In [4]:
def split_to_2_targets(dataset1, dataset2, outcome):
    target1 = dataset1[outcome]
    target2 = dataset2[outcome]
    return target1, target2

In [5]:
def train_test_split_data(dataset_X, dataset_y):
    return train_test_split(dataset_X, dataset_y, random_state=1, shuffle = True, stratify=dataset_y)

In [6]:
def create_decision_tree(max_depth=6):
    model = DecisionTreeClassifier(random_state=1, max_depth=max_depth)
    return model

In [7]:
def train_decision_tree(train_X, train_y, model):
    return model.fit(train_X, train_y)

In [8]:
def test_decision_tree(model, testcases):
    return model.predict(testcases)

In [9]:
def calculate_accuracy(testcases, prediction_result):
    return accuracy_score(testcases, prediction_result)

In [10]:
def calculate_similarity(prediction_result1, prediction_result2):
    return accuracy_score(prediction_result1, prediction_result2)

In [11]:
def number_of_disagreed_cases(testcases, prediction_result):
    wrong_proportion = 1 - calculate_accuracy(testcases, prediction_result)
    return round(len(testcases) * wrong_proportion)